In [9]:
import pandas as pd
from pandas import DataFrame
import os
from geopy import geocoders
from geopy.geocoders import GoogleV3
from config import gkey

adresses = ["2620 N Drake Ave, Chicago, Illinois 60647",
         "2622 N Drake Ave, Chicago, Illinois 60647",
         "2626 N Drake Ave, Chicago, Illinois 60647",
         "2630 N Drake Ave, Chicago, Illinois 60647",
         "2648 N Drake Ave, Chicago, Illinois 60647"]

In [10]:
df = DataFrame (adresses, columns=['Address'])
df

,Address
0,"2620 N Drake Ave, Chicago, Illinois 60647"
1,"2622 N Drake Ave, Chicago, Illinois 60647"
2,"2626 N Drake Ave, Chicago, Illinois 60647"
3,"2630 N Drake Ave, Chicago, Illinois 60647"
4,"2648 N Drake Ave, Chicago, Illinois 60647"


In [11]:
API_KEY = os.getenv("API1234")
g = GoogleV3(api_key=gkey)


loc_coordinates = []
loc_address = []
for address in df.Address:
    try:
        inputAddress = address
        location = g.geocode(inputAddress, timeout=15)
        loc_coordinates.append((location.latitude, location.longitude))
        loc_address.append(inputAddress)
    except Exception as e:
        print('Error, skipping address...', e)


df_geocodes = pd.DataFrame({'coordinate':loc_coordinates,'address':loc_address})
df_geocodes

,coordinate,address
0,"(41.9288822, -87.7161617)","2620 N Drake Ave, Chicago, Illinois 60647"
1,"(41.9289991, -87.7162002)","2622 N Drake Ave, Chicago, Illinois 60647"
2,"(41.9290774, -87.71623869999999)","2626 N Drake Ave, Chicago, Illinois 60647"
3,"(41.9291928, -87.7162002)","2630 N Drake Ave, Chicago, Illinois 60647"
4,"(41.92968949999999, -87.7162487)","2648 N Drake Ave, Chicago, Illinois 60647"


In [ ]:
import urllib.request, urllib.parse 
import requests
from requests.utils import requote_uri
import json
# Import google_streetview for the api module
import google_streetview.api


# Define parameters for street view api
params = [{
    'size': '600x300', # max 640x640 pixels
    'location': '41.9295582,-87.7159139',
    'heading': '90',
    'pitch': '-0.76',
    'key': gkey
}]

# Create a results object
results = google_streetview.api.results(params)

# Download images to directory 'downloads'
results.download_links('static/data/raw')

In [ ]:
# GRAB A SERIES OF STREET VIEW IMAGES FROM ADDRESSES PROVIDED
'''
myloc = "static/data/raw"
key = key = "&key=" + "gkey"
def GetStreet(Add,SaveLoc):
    base = "https://maps.googleapis.com/maps/api/streetview?size=1200x800&location="
    #MyUrl = base + urllib.parse.quote_plus(Add) + key  #added url encoding
    MyUrl = base + requote_uri(Add) + key  #added url encoding
    fi = Add + ".jpg"
    requests.get(MyUrl, os.path.join(SaveLoc,fi), stream=True, headers={'User-agent': 'Mozilla/5.0'})
  

Tests = ["2620 N Drake Ave, Chicago, Illinois 60647",
         "2622 N Drake Ave, Chicago, Illinois 60647",
         "2626 N Drake Ave, Chicago, Illinois 60647",
         "2630 N Drake Ave, Chicago, Illinois 60647",
         "2648 N Drake Ave, Chicago, Illinois 60647"]

for i in Tests:
 GetStreet(Add=i,SaveLoc=myloc)
''' 
  
    

In [ ]:

'''
# load and show an image with Pillow

# load the image
image = Image.open('static/data/test/brickornot.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()'''

In [ ]:
'''# load all images in a directory
loaded_images = list()
for filename in listdir('images'):
	# load image
	img_data = image.imread('images/' + filename)
	# store loaded image
	loaded_images.append(img_data)
	print('> loaded %s %s' % (filename, img_data.shape))'''


In [ ]:
%matplotlib inline
import PIL
from PIL import Image
from os import listdir
from matplotlib import image
from matplotlib import pyplot

# example of saving a grayscale version of a loaded image
from PIL import Image
# load the image
image = Image.open('static/data/test/brickornot.jpg')
# convert the image to grayscale
gs_image = image.convert(mode='L')
# report the size of the image
print(image.size)
# create a thumbnail and preserve aspect ratio
image.thumbnail((100,100))
# report the size of the thumbnail
print(image.size)
# save in jpeg format
gs_image.save('static/data/test_convert/brickornot.jpg')

In [ ]:
# load the image again and show it
image2 = Image.open('static/data/test_convert/brickornot.jpg')
# show the image
image2.show()

In [ ]:
'''# save as PNG format
image.save('opera_house.png', format='PNG')
# load the image again and inspect the format
image2 = Image.open('opera_house.png')
print(image2.format)'''